# Parse HLS persons notices

The overall goal is to "clean" notices by removing html, links, abreviations etc from the notices. 

In [1]:
import pandas as pd
import re
import geovpylib.database as db
from gmpykit.ipython import infos
import gmpykit as kit

db.connect_yellow('switzerland_and_beyond')

[DB] Requests will not be executed
[DB] Connecting to YELLOW database "switzerland_and_beyond" ... Connected!


## Fetch the data

In [2]:
persons = db.query('select * from hls.person')
infos(persons)

Shape:  (24966, 5) - extract:


,id,url,name,notice_html,notice
0,5588,https://hls-dhs-dss.ch/fr/articles/003039,"Jeker, Amanz","<span class=""hls-dnais"" locale=""fr"">10.1.1817<...",None
1,14,https://hls-dhs-dss.ch/fr/articles/025222,"König, Johann","<span class=""hls-dnais"" locale=""fr"">28.4.1639<...",None
2,23,https://hls-dhs-dss.ch/fr/articles/024083,"Muralt, Johannes von","<span class=""hls-dnais"" locale=""fr"">18.7.1877<...",None
3,31,https://hls-dhs-dss.ch/fr/articles/023373,"Burckhardt, Emanuel","<span class=""hls-dnais"" locale=""fr"">25.11.1744...",None
4,39,https://hls-dhs-dss.ch/fr/articles/024670,"Forcart-Respinger, Emily","<span class=""hls-dnais"" locale=""fr"">9.11.1886<...",None


## Update the table

In [3]:
if "notice" not in persons.columns:
    db.connect_yellow('switzerland_and_beyond', execute=True)
    db.execute('alter table hls.person add notice varchar;')
    db.connect_yellow('switzerland_and_beyond', execute=False)

## Clean the HTML

In [4]:
example_name = persons.iloc[0]['name']
example_url = persons.iloc[0]['url']
example_notice = persons.iloc[0]['notice_html']
print(example_name)
print(example_url)
print(kit.wrap(example_notice))

Jeker, Amanz
https://hls-dhs-dss.ch/fr/articles/003039
<span class="hls-dnais" locale="fr">10.1.1817</span> à Seewen (SO), <span class="hls-ddec" locale="fr">11.2.1875</span> à Soleure, cath., puis
cath.-chr., de Mümliswil-Ramiswil. Fils d'Urs, agriculteur au Rechtenberg (comm. Seewen). <span class="hls-alli" locale="fr">​</span>Albertine Hofer,
fille d'Urs Jakob, aubergiste et député au Grand Conseil soleurois, de Mümliswil-Ramiswil. Gymnase à Fribourg et Soleure, études de droit à Fribourg
et Munich. Agriculteur sur le domaine de Rechtenberg. Député radical au Grand Conseil soleurois et juge pénal à Dornach (1839-1850), conseiller d'Etat
représentant les paysans et partisan de Joseph Wilhelm Viktor Vigier von Steinbrugg (1848-1850), préfet d'Olten-Gösgen (1850-1861), à nouveau
conseiller d'Etat (1861-1866 et 1869-1875, landamman en 1870), conseiller aux Etats (1862-1874). Durant le Kulturkampf, L. prit la parole lors de la
manifestation anti-catholique-romaine d'Arlesheim (1873). Col

In [168]:
def get_name(name):
    try: comma_index = name.index(', ')
    except: return name

    first = name[0:comma_index]
    second = name[comma_index + 1:]
    
    return second.strip() + ' ' + first.strip()

def is_date_format(string):
    splits = string.split('.')
    try:
        day = int(splits[0])
        month = int(splits[1])
        year = int(splits[2])
        return True
    except:
        return False


def handle_baptisme(notice):
    begin_words = '<span class="hls-bapt" locale="fr">'
    end_words = '</span>'
    try:
        begin = notice.index(begin_words)
        end = notice.index(end_words, begin) + len(end_words)
        origin = notice[begin:end]
        result = origin.replace(begin_words, '').replace(end_words, '')
        if is_date_format(result):
            result = 'baptisé le ' + result
        else:
            result = 'baptisé ' + result
        return notice.replace(origin, result)
    except: return notice


def handle_birthdate(notice):
    begin_words = '<span class="hls-dnais" locale="fr">'
    end_words = '</span>'
    try:
        begin = notice.index(begin_words)
        end = notice.index(end_words, begin) + len(end_words)
        origin = notice[begin:end]
        result = origin.replace(begin_words, '').replace(end_words, '')
        if is_date_format(result):
            result = 'naît le ' + result
        else:
            result = 'naît ' + result
        return notice.replace(origin, result)
    except: return notice


def handle_deathdate(notice):
    begin_words = '<span class="hls-ddec" locale="fr">'
    end_words = '</span>'
    try:
        begin = notice.index(begin_words)
        end = notice.index(end_words, begin) + len(end_words)
        origin = notice[begin:end]
        result = origin.replace(begin_words, '').replace(end_words, '')
        if is_date_format(result):
            result = 'meurt le ' + result
        else:
            result = 'meurt ' + result
        return notice.replace(origin, result)
    except: return notice


def handle_sepulture(notice):
    begin_words = '<span class="hls-sepu" locale="fr">'
    end_words = '</span>'
    try:
        begin = notice.index(begin_words)
        end = notice.index(end_words, begin) + len(end_words)
        origin = notice[begin:end]
        result = origin.replace(begin_words, '').replace(end_words, '')
        if is_date_format(result):
            result = 'enterré le' + result
        else:
            result = 'enterré ' + result
        return notice.replace(origin, result)
    except: return notice


def handle_cit(notice):
    begin_words = '<span class="hls-cit" locale="fr">'
    end_words = '</span>'
    try:
        begin = notice.index(begin_words)
        end = notice.index(end_words, begin) + len(end_words)
        origin = notice[begin:end]
        result = origin.replace(begin_words, '').replace(end_words, '')
        return notice.replace(origin, result)
    except: return notice

def handle_cit2(notice):
    begin_words = '<span class="hls-cit2" locale="fr">'
    end_words = '</span>'
    try:
        begin = notice.index(begin_words)
        end = notice.index(end_words, begin) + len(end_words)
        origin = notice[begin:end]
        result = origin.replace(begin_words, '').replace(end_words, '')
        return notice.replace(origin, result)
    except: return notice


def handle_mariage(notice):
    # Zero.1
    to_find = '. <span class="hls-alli" locale="fr"></span>avant'
    notice = notice.replace(to_find, '. Marié(e) avant')
    # Zero.2
    to_find = '. <span class="hls-alli" locale="fr"></span>après'
    notice = notice.replace(to_find, '. Marié(e) après')

    # First case without date, begining of sentence
    to_find = '. <span class="hls-alli" locale="fr"></span>'
    notice = notice.replace(to_find, '. Marié(e) à ')
    
    # First case without date, begining of sentence
    to_find = '. <span class="hls-alli" locale="fr"></span>'
    notice = notice.replace(to_find, '. Marié(e) à ')

    # Second case without date, middle of sentence
    to_find = '<span class="hls-alli" locale="fr"></span>'
    notice = notice.replace(to_find, 'marié(e) à')

    # Third case with date
    begin_words = '<span class="hls-alli" locale="fr">'
    end_words = '</span>'
    try:
        begin = notice.index(begin_words)
        end = notice.index(end_words, begin) + len(end_words)
        origin = notice[begin:end]
        result = origin.replace(begin_words, '').replace(end_words, '')
        result = 'Marié(e) (' + result + ') à'
        notice = notice.replace(origin, result)
    except: pass

    return notice


def handle_links(notice):
    begin_words = '<span class="wikilink">'
    end_words = '</span>'

    while 'href' in notice:
        try:
            begin = notice.index(begin_words)
            end = notice.index(end_words, begin) + len(end_words)
            href_begin = notice.index('"', notice.index('href')) + 1
            href_end = notice.index('"', href_begin)
            origin = notice[begin:end]

            link_content_begin = origin.index('>', origin.index('<a ')) + 1
            link_content_end = origin.index('</a>')
            link_content = origin[link_content_begin:link_content_end].strip()
            link_content = link_content.replace('-&gt;', '->')
            if not link_content == '->': 
                link_content += ' (->'
                suffix = ')'
            else: suffix = ''

            notice = notice.replace(origin, link_content + "https://hls-dhs-dss.ch" + notice[href_begin:href_end] + suffix)
        except: pass
    
    return notice


def format_begin_end(notice):
    return (notice[0:1].upper() + notice[1:]).strip()


def expand_name(notice, name):
    unique_caps_with_dot = re.findall(r'\s([A-Z])\.', notice)
    for caps in unique_caps_with_dot:
        notice = notice.replace(f' {caps}. ', ' ' + name + ' ')
        notice = notice.replace(f' {caps}.,', ' ' + name + ' ')
    return notice


def expand_shortcuts(notice):
    notice = notice.replace('cath.-chr.', 'catholique chrétien')
    notice = notice.replace('cath.', 'catholique')
    notice = notice.replace('prot.', 'protestant')
    notice = notice.replace('comm.', 'commune de')
    notice = notice.replace('auj.', 'aujourd\'hui')
    return notice

def handle_power_letters(notice):
    notice = notice.replace('<sup><span locale="fr">e</span></sup>', 'e')
    return notice

def replace_chars(notice):
    notice = notice.replace('&nbsp;', ' ')
    notice = notice.replace('&ZeroWidthSpace;', ' ')
    notice = notice.replace('\u200b', '')
    return notice

def handle_stanger_words(notice):
    begin_words = '<em><span locale="fr">'
    end_words = '</span></em>'

    while '<em>' in notice:
        try:
            begin = notice.index(begin_words)
            end = notice.index(end_words, begin) + len(end_words)
            origin = notice[begin:end]
            result = origin.replace(begin_words, '').replace(end_words, '')
            notice = notice.replace(origin, result)
        except: notice = notice
    return notice

def handle_stanger_words2(notice):
    begin_words = '<sup><span locale="fr">'
    end_words = '</span></sup>'

    while '<sup>' in notice:
        try:
            begin = notice.index(begin_words)
            end = notice.index(end_words, begin) + len(end_words)
            origin = notice[begin:end]
            result = origin.replace(begin_words, '').replace(end_words, '')
            notice = notice.replace(origin, result)
        except: notice = notice
    return notice

def handle_numbers(notice):
    return notice


def handle_notice(name, notice):
    name = get_name(name)
    notice = kit.remove_bin_chars(notice)
    notice = replace_chars(notice)
    notice = handle_stanger_words(notice)
    notice = handle_stanger_words2(notice)
    notice = handle_cit(notice)
    notice = handle_cit2(notice)
    notice = handle_birthdate(notice)
    notice = handle_baptisme(notice)
    notice = handle_deathdate(notice)
    notice = handle_sepulture(notice)
    notice = handle_mariage(notice)
    notice = handle_links(notice)
    notice = handle_power_letters(notice)
    notice = format_begin_end(notice)
    notice = expand_shortcuts(notice)
    notice = expand_name(notice, name)
    notice = handle_numbers(notice)

    return notice.replace('  ', ' ')

In [177]:
for i, row in persons.sample(1).iterrows():
    print(row['name'])
    print(row['url'], i)
    print(kit.wrap(handle_notice(row['name'], row['notice_html']), 100))


Wildberger, Hans
https://hls-dhs-dss.ch/fr/articles/010917 21429
Naît le 2.1.1910 à Neunkirch, meurt le 25.6.1986 à Zurich, protestant, de Neunkirch. Fils de Johann
Georg Wildberger, agriculteur, et d'Elise née Wildberger. Marié(e) (1934) à Hedwig Kägi. Hans
Wildberger étudia la théologie à Zurich, Marbourg et Bonn. Pasteur (->https://hls-dhs-
dss.ch/fr/articles/011522/2011-02-01/) à Wilchingen (1933-1939), puis à Lucerne (1939-1951), il fut
ensuite professeur d'Ancien Testament et d'histoire générale des religions à l'Université de Zurich
(->https://hls-dhs-dss.ch/fr/articles/010977/2013-01-28/) (1951-1975). Il fit partie de l'autorité
théologique en matière d'examens pastoraux instituée par le concordat des Eglises protestantes de
Suisse alémanique (1959-1979, président pendant 18 ans). Marqué sur le plan scientifique par
l'enseignement de son principal professeur, Ludwig Köhler (->https://hls-dhs-
dss.ch/fr/articles/010710/2008-10-28/), Wildberger fut influencé tout au long de sa vi

In [113]:
row = persons.loc[i]
print(row['name'])
print(row['url'])
print(kit.wrap(handle_notice(row['name'], row['notice_html']), 100))


Zweifel, Ludwig
https://hls-dhs-dss.ch/fr/articles/029310
Naît le 30.10.1724 à Ennetlinth (aujourd'hui commune de Glaris Sud), meurt le 18.3.1779 à Netstal
(aujourd'hui commune de Glaris), protestant, de Glaris et Linthal (aujourd'hui commune de Glaris
Sud). Fils de Ludwig, membre du Conseil, maître de la batellerie et administrateur paroissial, et de
Katharina née Ludwig Zweifel Marié(e) (1749) à Maria Magdalena Weber, fille de Fridolin, lieutenant
et papetier. Ludwig Zweifel s'établit en 1752 à Netstal où il acheta en 1754 un moulin à grain qui
resta dans la famille jusqu'en 1827. En 1755, il devint président du Tagwen de Netstal. Il acheta en
1764 à son beau-frère Heinrich Weber le moulin à papier exploité par trois générations de Weber,
avec monopole de l'achat des vieux chiffons dans le canton de Glaris, matière première pour la
production du papier. Ludwig Zweifel obtint encore du Conseil une interdiction de toute exportation
de chiffons hors du canton. Il fonda ainsi la dynastie

In [95]:
to_check = [13083, 15831, 16789]

'<span class="hls-dnais" locale="fr">6.3.1827</span> à Sursee, <span class="hls-ddec" locale="fr">30.8.1894</span> à Sursee, cath., de Sursee. Fils de Franz Xaver, agriculteur et juge de district. <span class="hls-alli" locale="fr">\u200b</span>Maria Leu, fille de <span class="wikilink"><a class="rtxt" title="Josef Leu" target="_blank" rel="noopener noreferrer" href="/fr/articles/013347/2016-03-03/">Josef Leu</a></span> d\'Ebersol. Gymnase à Lucerne, collège à Fribourg, début d\'études à Munich et Fribourg-en-Brisgau. Officier lors de la guerre du Sonderbund. Après une formation agricole, B. dirigea le grand domaine de Beckenhof à Sursee. Cofondateur en 1859 de la Société lucernoise des paysans (président en 1860). Membre du Conseil de la commune bourgeoise (1864-1867), juge de district (1868-1881), député lucernois (1871-1891), conseiller national (1869-1894). B. fut l\'un des premiers hommes politiques représentant la paysannerie; il proposa notamment une loi contre l\'endettement de